In [1]:
import datetime as dt
from utility import (
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_qtfdnav,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标债券型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(report_dates_begin, 203)

In [4]:
# 获取相关组合情况

asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100

In [5]:
last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

In [6]:
agg_port = pd.concat([last_portfolio, last_4_cov_portfolio, last_4_stk_portfolio], axis=1).reset_index()
agg_port = pd.merge(agg_port, fd_typeclass(agg_port.SECURITYID.tolist(), current_date))
basic_info = fd_basicinfo(agg_port.SECURITYID.tolist())
agg_port = pd.merge(agg_port, basic_info)

In [7]:
agg_port.L3NAME.unique()

array(['普通债券型基金(一级A类)', '普通债券型基金(一级B/C类)', '普通债券型基金(二级A类)',
       '普通债券型基金(二级B/C类)', '中短期标准债券型基金', '偏债型基金', '普通债券型基金(可投转债A类)',
       '普通债券型基金(可投转债B类)', '指数债券型基金(A类)', '长期标准债券型基金(A类)', 'QDII债券基金',
       '可转换债券型基金(A类)', '可转换债券型基金(B/C类)', '指数债券型基金(B/C类)',
       '长期标准债券型基金(B/C类)', '封闭式普通债券型基金(可投转债)(B/C类)', '短期理财债券型基金(A类)',
       '短期理财债券型基金(B/C类)', '封闭式长期标准债券型基金(A类)', '封闭式长期标准债券型基金(B/C类)',
       '保本型基金', '封闭式普通债券型基金(可投转债)(A类)', '封闭式普通债券型基金(一级)(A类)',
       '债券型分级子基金(优先份额)', '债券型分级子基金(进取份额)', '封闭式普通债券型基金(二级)(A类)',
       '封闭式普通债券型基金(二级)(B/C类)', '封闭式普通债券型基金(二级)', '封闭式债券型分级子基金(优先份额)',
       '封闭式债券型分级子基金(进取份额)', '灵活配置型基金(股票上限95%)（A类）',
       '灵活配置型基金(股票上限95%)（B/C类）'], dtype=object)

In [8]:
# 分类
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 5) & (agg_port["CONVBDRTO_TO_BDRTO"] < 80) & (agg_port["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["EQUITYINVERTO"] >= 0.1) & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["L2NAME"] == "指数债券型基金") & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
agg_port.loc[pd.isnull(agg_port["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"
agg_port = agg_port.sort_values("SECURITYID").dropna(subset=["债券型（子类）"])

In [9]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,871,871,854,860,871,871,871,871,871,871,871,636,871,871,871
可投股票型债券基金,386,386,386,386,386,386,386,386,386,386,386,332,386,386,386
可投转债型债券基金,111,111,111,111,111,111,111,111,111,111,111,95,111,111,111
可转债基金,42,42,42,42,42,42,42,42,42,42,42,36,42,42,42
短期纯债型基金,47,47,47,47,47,47,47,47,47,47,47,46,47,47,47
被动指数型债券基金,16,16,16,16,16,16,16,16,16,16,16,11,16,16,16


## 1.1 Put it all totether

In [10]:
def create_bond_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 203)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
    asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100
    last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
    last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
    last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, last_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_stk_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_cov_portfolio, on="SECURITYID", how="left").drop_duplicates()
    
    df.loc[(df["BDRTO"] >= 80) & (df.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 5) & (df["CONVBDRTO_TO_BDRTO"] < 80) & (df["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["EQUITYINVERTO"] >= 0.1) & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["L2NAME"] == "指数债券型基金") & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
    df.loc[pd.isnull(df["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"
    
    return df.sort_values("SECURITYID").dropna(subset=["债券型（子类）"])

In [11]:
%%time

df = create_bond_fund_info("20211111")#.groupby( "债券型（子类）")

Wall time: 9.48 s


In [12]:
%%time

create_bond_fund_info("20211111").groupby( "债券型（子类）").count()

Wall time: 8.24 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,BDRTO,EQUITYINVERTO,CONVBDRTO_TO_BDRTO
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,912,912,674,912,912,912,912,912,912,912,912,912,872,861,855
可投股票型债券基金,386,386,332,386,386,386,386,386,386,386,386,386,386,386,386
可投转债型债券基金,111,111,95,111,111,111,111,111,111,111,111,111,111,111,111
可转债基金,42,42,36,42,42,42,42,42,42,42,42,42,42,42,42
短期纯债型基金,47,47,46,47,47,47,47,47,47,47,47,47,47,47,47
被动指数型债券基金,16,16,11,16,16,16,16,16,16,16,16,16,16,16,16


In [13]:
ddf = create_bond_fund_info("20211111")

In [14]:
ddf

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,BDRTO,EQUITYINVERTO,CONVBDRTO_TO_BDRTO,债券型（子类）
0,1030000011,华夏债券投资基金,华夏债券,001001,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),89.97,2.5125,26.361539,可投股票型债券基金
1,1030000012,华夏债券投资基金,华夏债券,001003,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.2,普通债券型基金(一级B/C类),113.82,2.0925,30.838271,可投股票型债券基金
2,1030000014,华夏希望债券型证券投资基金,华夏希望债券,001011,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.3,普通债券型基金(二级A类),84.69,7.8500,0.939904,可投股票型债券基金
3,1030000015,华夏希望债券型证券投资基金,华夏希望债券,001013,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.4,普通债券型基金(二级B/C类),94.13,6.4125,2.208407,可投股票型债券基金
4,1030000022,国泰金龙债券证券投资基金,国泰金龙债券,020002,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),130.35,1.0475,11.735895,可投股票型债券基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,2030008316,金鹰持久增利债券型证券投资基金(LOF),金鹰持久增利债券(LOF),162105,LOF,稳健成长型,3,债券基金,3.2,普通债券型基金,3.2.4,普通债券型基金(二级B/C类),84.99,19.0050,70.112516,可投股票型债券基金
1488,2030008318,浦银安盛稳健增利债券型证券投资基金(LOF),浦银安盛稳健增利债券(LOF),166401,LOF,稳健成长型,3,债券基金,3.2,普通债券型基金,3.2.2,普通债券型基金(一级B/C类),123.34,0.0000,1.108862,中长期纯债型基金
1149,2030008417,金鹰元盛债券型发起式证券投资基金(LOF),金鹰元盛债券(LOF),162108,LOF,稳健成长型,3,债券基金,3.2,普通债券型基金,3.2.6,普通债券型基金(可投转债B类),104.85,0.0000,0.046836,中长期纯债型基金
1199,2030011710,招商双债增强债券型证券投资基金(LOF),招商双债增强(LOF),161716,LOF,稳健成长型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),90.45,0.0550,1.440198,中长期纯债型基金


# 2. 基金指数
--------------------

In [15]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 fd_qtfdnav 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "REPAIRUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [16]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_bond_fund_info(trade_dt)
    pre_nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("债券型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [17]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.").to_excel("030_债券型基金_bak.xlsx")

In [18]:
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.")

债券型（子类）,中长期纯债型基金,可投股票型债券基金,可投转债型债券基金,可转债基金,短期纯债型基金,被动指数型债券基金
level_0,,,,,,
20150430,0.037476,0.071453,0.037418,0.198128,NaN,0.018172
20150731,0.018956,0.001535,0.013327,-0.127380,NaN,0.007441
20151030,0.022873,0.018455,0.022593,0.003761,NaN,0.023658
20160129,0.007949,-0.011013,0.008938,-0.130587,NaN,0.016942
20160429,0.001275,0.006342,0.002166,-0.013352,NaN,0.000653
20160729,0.023722,0.021698,0.020302,-0.010029,NaN,0.020498
20161031,0.014215,0.016430,0.014736,0.042255,NaN,0.010548
20170126,-0.017846,-0.020942,-0.020554,-0.058226,NaN,-0.022313
20170428,-0.000509,-0.003125,-0.004312,-0.021898,NaN,-0.006358
